In [1]:
import gymnasium as gym
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from stable_baselines3 import DQN, PPO, A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from stable_baselines3.common.callbacks import (
    EvalCallback,
    StopTrainingOnRewardThreshold,
    BaseCallback,
)
from stable_baselines3.common.callbacks import ProgressBarCallback
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
# Create the LunarLander-v3 environment
env = gym.make(
    "LunarLander-v3",
    # continuous=False,
    # gravity=-10.0,
    # enable_wind=False,
    # wind_power=15.0,
    # turbulence_power=1.5,
    render_mode="rgb_array",
)

In [3]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)

eval_callback = EvalCallback(
    env,
    best_model_save_path="../logs/exercise_2/a2c/",
    log_path="../logs/exercise_2/a2c/",
    eval_freq=5_000,
    deterministic=True,
    render=False,
    n_eval_episodes=50,
    callback_on_new_best=stop_callback,
)

In [4]:
# Create A2C model
model = A2C(
    "MlpPolicy",
    env,
    learning_rate=0.0006997150606687543,
    gamma=0.9987615431097223,
    ent_coef=0.014358534695948685,
    vf_coef=0.9222159161633121,
    n_steps=5,
    policy_kwargs=dict(net_arch=dict(pi=[128], vf=[256])),
    seed=42,
    verbose=0,
    tensorboard_log="../logs/exercise_2/a2c/a2c_tensorboard/",
    device="cpu",
)

In [5]:
# Train model on the environment
TRAINING_TIMESTEPS = 500_000
model.learn(
    total_timesteps=TRAINING_TIMESTEPS, callback=[eval_callback, ProgressBarCallback()]
)

Output()

/home/javier/.cache/pypoetry/virtualenvs/deep-reinforcement-learning-gymnasium-u3px5S1O-py3.13/lib/python3.13/site-
packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a 
``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to
modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(

Eval num_timesteps=5000, episode_reward=-694.86 +/- 213.07

Episode length: 501.00 +/- 191.07

New best mean reward!

...

Episode length: 1000.00 +/- 0.00

Eval num_timesteps=495000, episode_reward=-107.12 +/- 33.14

Episode length: 1000.00 +/- 0.00

Eval num_timesteps=500000, episode_reward=-164.30 +/- 50.71

Episode length: 977.48 +/- 68.91

In [9]:
model = A2C.load("../logs/exercise_2/a2c/best_model.zip")

/home/javier/.cache/pypoetry/virtualenvs/deep-reinforcement-learning-gymnasium-u3px5S1O-py3.13/lib/python3.13/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run A2C on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


In [10]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=50)
print(f"Mean reward: {mean_reward} +/- {std_reward}")

/home/javier/.cache/pypoetry/virtualenvs/deep-reinforcement-learning-gymnasium-u3px5S1O-py3.13/lib/python3.13/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: 177.99962731934156 +/- 95.19826463320929


In [11]:
MAX_STEPS = 1_000

observation, info = env.reset(seed=42)
frames = []
step_count = 0

# for step in range(MAX_STEPS):
while True:
    frame = env.render()
    frames.append(frame)

    action, _ = model.predict(observation, deterministic=True)
    observation, reward, terminated, truncated, info = env.step(action)
    step_count += 1

    if terminated or truncated:
        print(
            f"Episode finished after {step_count} steps ({"truncated" if truncated else "terminated"})"
        )
        break

env.close()

print(f"Final Step: {step_count}")
print(f"Number of Frames: {len(frames)}")

Episode finished after 456 steps (terminated)
Final Step: 456
Number of Frames: 456


In [12]:
from IPython.display import HTML
from base64 import b64encode
import os

# Create a video from the frames
video_filename = "../videos/lunarlander_a2c.mp4"
compressed_path = "../videos/lunarlander_a2c_compressed.mp4"
height, width, _ = frames[0].shape

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
video = cv2.VideoWriter(video_filename, fourcc, 30.0, (width, height))

for frame in frames:
    video.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
video.release()

print(f"Video guardado como {video_filename}")

os.system(f"rm {compressed_path}")
# Compressed video path
os.system(f"ffmpeg -i {video_filename} -vcodec libx264 {compressed_path}")
os.system(f"rm {video_filename}")
os.system(f"mv {compressed_path} {video_filename}")

# Show video
mp4 = open(video_filename, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(
    """
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>"""
    % data_url
)

Video guardado como ../videos/lunarlander_a2c.mp4


rm: cannot remove '../videos/lunarlander_a2c_compressed.mp4': No such file or directory
ffmpeg version n7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.1 (GCC) 20250207
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-frei0r --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libdvdnav --enable-libdvdread --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgsm --enable-libharfbuzz --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libplacebo --enable-libpulse --enable-librav1e --enable-librsvg --enable-librubberband --enable-libsnappy --enab